# Run OpenQuake scenarios sensitivity analysis

## Configuration and user input

In [6]:
# Import dependencies
import os, glob
import oq_run_many as oq
from oq_parse_gmfs import oq_get_gmfs, write_gmf_csv


# Event name
event = '20030521_M6.8_Boumerdes'

# Indicate ruptures to be considered in calculations
ruptures_to_include = ['USGS', 'BelabbesEtAl2009', 'SemmaneEtAl2005']

# Ground Motion Models or GMPE's to include.
gmm_sources_to_include = ['GEM', 'USGS']

## Folders and files

In [7]:
# Find all rupture files
all_rups = glob.glob(os.path.join('..', '*', event,
                                  'Rupture',
                                  'earthquake_rupture_model_*'))

# Verify that ruptures exist in folder
ruptures = []
for author in ruptures_to_include:
    exist = False 
    for rup in all_rups:
        if author in rup:
            exist = True
            ruptures.append(rup)
    if not exist:
        raise ValueError(f'No rupture found for author {author}!')

# OQ rupture paths
oq_rups = [f'../Rupture/{os.path.basename(rup)}' for rup in ruptures]

# OQ gmlt paths
oq_gmlt = [f'gmpe_logic_tree_{gmlt}.xml' for gmlt in gmm_sources_to_include]

# Event folder
country = ruptures[0].split(os.sep)[1]
root_folder = ruptures[0][:ruptures[0].find('Rupture')]
oq_folder = os.path.join(root_folder, 'OpenQuake_gmfs')
oq_sens = os.path.join(oq_folder, 'Sensitivity')

# Create Sensitivity folder
if os.path.isdir(oq_sens):
    print('Sensitivity folder already exists\n')
else:
    os.mkdir(oq_sens)
    print('Sensitivity folder created\n')


print('GMLT paths for OQ:', oq_gmlt)
print('Rupture paths for OQ:')
oq_rups


Sensitivity folder already exists

GMLT paths for OQ: ['gmpe_logic_tree_GEM.xml', 'gmpe_logic_tree_USGS.xml']
Rupture paths for OQ:


['../Rupture/earthquake_rupture_model_USGS.xml',
 '../Rupture/earthquake_rupture_model_BelabbesEtAl2009.xml',
 '../Rupture/earthquake_rupture_model_SemmaneEtAl2005.xml']

## Run calculation without conditioning the GMFs

### Stations: None

In [8]:
# Job.ini to be use in the sensitivity analysis
job_ini = os.path.join(oq_folder, 'job_stations_none.ini')

# Run OpenQuake calculations
jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# For each calculation, save gmfs in csv (only for PGA)
for job in jobs:
    calc_id = job.calc_id
    df, txt = oq_get_gmfs(calc_id)
    out_df = write_gmf_csv(df, 'PGA', comment=txt, 
                           out_path=oq_sens, suffix=calc_id)

[2023-12-21 10:35:37 #468 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-12-21 10:35:37 #468 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:35:38 #468 WARNING] Using 16 processpool workers
[2023-12-21 10:35:38 #468 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:35:38 #468 INFO] Checksum of the inputs: 790354818 (total size 425.82 KB)
[2023-12-21 10:35:38 #468 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:35:38 #468 INFO] Read N=8131 hazard sites and L=1 hazard levels
[2023-12-21 10:35:38 #468 INFO] min_iml=[1.e-10]
[2023-12-21 10:35:38 #468 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:35:38 #468 INFO] Reading 1 ruptures
[2023-12-21 10:35:38 #468 INFO] Affected sites = 8131.0 per rupture
[2023-12-21 10:35:38 #468 INFO] totw = 3
[2023-12-21 10:35:42 #468 INFO] gen_

  id | name
1078 | Events
1079 | Full Report
1080 | Ground Motion Fields
1081 | Realizations


[2023-12-21 10:35:47 #469 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-12-21 10:35:47 #469 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:35:49 #469 WARNING] An error occurred while calling https://api.openquake.org/engine/latest to check if the installed version of the engine is up to date.
[2023-12-21 10:35:49 #469 WARNING] Using 16 processpool workers
[2023-12-21 10:35:49 #469 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:35:49 #469 INFO] Checksum of the inputs: 632533997 (total size 425.66 KB)
[2023-12-21 10:35:49 #469 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:35:49 #469 INFO] Read N=8131 hazard sites and L=1 hazard levels
[2023-12-21 10:35:49 #469 INFO] min_iml=[1.e-10]
[2023-12-21 10:35:49 #469 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:35:49 #469 

  id | name
1082 | Events
1083 | Full Report
1084 | Ground Motion Fields
1085 | Realizations


[2023-12-21 10:35:58 #470 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-12-21 10:35:58 #470 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:35:59 #470 WARNING] An error occurred while calling https://api.openquake.org/engine/latest to check if the installed version of the engine is up to date.
[2023-12-21 10:35:59 #470 WARNING] Using 16 processpool workers
[2023-12-21 10:35:59 #470 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:35:59 #470 INFO] Checksum of the inputs: 84784208 (total size 425.67 KB)
[2023-12-21 10:35:59 #470 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:35:59 #470 INFO] Read N=8131 hazard sites and L=1 hazard levels
[2023-12-21 10:35:59 #470 INFO] min_iml=[1.e-10]
[2023-12-21 10:35:59 #470 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:35:59 #470 I

...

  id | name
1090 | Events
1091 | Full Report
1092 | Ground Motion Fields
1093 | Realizations


[2023-12-21 10:36:18 #472 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-12-21 10:36:19 #472 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:36:19 #472 WARNING] Using 16 processpool workers
[2023-12-21 10:36:19 #472 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:36:19 #472 INFO] Checksum of the inputs: 1712533847 (total size 426.1 KB)
[2023-12-21 10:36:19 #472 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:36:19 #472 INFO] Read N=8131 hazard sites and L=1 hazard levels
[2023-12-21 10:36:19 #472 INFO] min_iml=[1.e-10]
[2023-12-21 10:36:19 #472 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:36:19 #472 INFO] Reading 1 ruptures
[2023-12-21 10:36:19 #472 INFO] Affected sites = 8131.0 per rupture
[2023-12-21 10:36:19 #472 INFO] totw = 3
[2023-12-21 10:36:23 #472 INFO] gen_

  id | name
1094 | Events
1095 | Full Report
1096 | Ground Motion Fields
1097 | Realizations


[2023-12-21 10:36:29 #473 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-12-21 10:36:29 #473 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:36:29 #473 WARNING] Using 16 processpool workers
[2023-12-21 10:36:29 #473 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:36:29 #473 INFO] Checksum of the inputs: 4058460602 (total size 426.1 KB)
[2023-12-21 10:36:29 #473 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:36:30 #473 INFO] Read N=8131 hazard sites and L=1 hazard levels
[2023-12-21 10:36:30 #473 INFO] min_iml=[1.e-10]
[2023-12-21 10:36:30 #473 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:36:30 #473 INFO] Reading 1 ruptures
[2023-12-21 10:36:30 #473 INFO] Affected sites = 8131.0 per rupture
[2023-12-21 10:36:30 #473 INFO] totw = 3
[2023-12-21 10:36:34 #473 INFO] gen_

  id | name
1098 | Events
1099 | Full Report
1100 | Ground Motion Fields
1101 | Realizations
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_468.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_469.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_470.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_471.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_472.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_473.csv


### Stations: Seismic

In [9]:
# Job.ini to be use in the sensitivity analysis
job_ini = os.path.join(oq_folder, 'job_stations_seismic.ini')

# Run OpenQuake calculations
jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# For each calculation, save gmfs in csv (only for PGA)
for job in jobs:
    calc_id = job.calc_id
    df, txt = oq_get_gmfs(calc_id)
    out_df = write_gmf_csv(df, 'PGA', comment=txt, 
                           out_path=oq_sens, suffix=calc_id)

[2023-12-21 10:36:39 #474 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-12-21 10:36:40 #474 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:36:41 #474 WARNING] An error occurred while calling https://api.openquake.org/engine/latest to check if the installed version of the engine is up to date.
[2023-12-21 10:36:41 #474 WARNING] Using 16 processpool workers
[2023-12-21 10:36:41 #474 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:36:41 #474 INFO] Checksum of the inputs: 1018325640 (total size 436.08 KB)
[2023-12-21 10:36:41 #474 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:36:41 #474 INFO] Read N=8280 hazard sites and L=1 hazard levels
[2023-12-21 10:36:41 #474 INFO] Reading station data from /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/s

  id | name
1102 | Events
1103 | Full Report
1104 | Ground Motion Fields
1105 | Realizations


[2023-12-21 10:38:29 #475 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-12-21 10:38:29 #475 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:38:30 #475 WARNING] Using 16 processpool workers
[2023-12-21 10:38:30 #475 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:38:30 #475 INFO] Checksum of the inputs: 1135362803 (total size 435.92 KB)
[2023-12-21 10:38:30 #475 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:38:30 #475 INFO] Read N=8280 hazard sites and L=1 hazard levels
[2023-12-21 10:38:30 #475 INFO] Reading station data from /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/stationlist_seismic.csv
[2023-12-21 10:38:30 #475 INFO] min_iml=[1.e-10]
[2023-12-21 10:38:30 #475 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:38:3

  id | name
1106 | Events
1107 | Full Report
1108 | Ground Motion Fields
1109 | Realizations


[2023-12-21 10:40:18 #476 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-12-21 10:40:18 #476 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:40:18 #476 WARNING] Using 16 processpool workers
[2023-12-21 10:40:18 #476 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:40:18 #476 INFO] Checksum of the inputs: 53953366 (total size 435.93 KB)
[2023-12-21 10:40:18 #476 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:40:19 #476 INFO] Read N=8280 hazard sites and L=1 hazard levels
[2023-12-21 10:40:19 #476 INFO] Reading station data from /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/stationlist_seismic.csv
[2023-12-21 10:40:19 #476 INFO] min_iml=[1.e-10]
[2023-12-21 10:40:19 #476 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:40:19 

...

  id | name
1114 | Events
1115 | Full Report
1116 | Ground Motion Fields
1117 | Realizations


[2023-12-21 10:45:34 #478 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-12-21 10:45:34 #478 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:45:35 #478 WARNING] Using 16 processpool workers
[2023-12-21 10:45:35 #478 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:45:35 #478 INFO] Checksum of the inputs: 2809905230 (total size 436.36 KB)
[2023-12-21 10:45:35 #478 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:45:35 #478 INFO] Read N=8280 hazard sites and L=1 hazard levels
[2023-12-21 10:45:35 #478 INFO] Reading station data from /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/stationlist_seismic.csv
[2023-12-21 10:45:35 #478 INFO] min_iml=[1.e-10]
[2023-12-21 10:45:35 #478 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:45:3

  id | name
1118 | Events
1119 | Full Report
1120 | Ground Motion Fields
1121 | Realizations


[2023-12-21 10:49:07 #479 INFO] nicolepaul@Nicoles-MacBook-Pro.local running /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-12-21 10:49:07 #479 INFO] Using engine version 3.19.0-gitea3782ec67
[2023-12-21 10:49:08 #479 WARNING] Using 16 processpool workers
[2023-12-21 10:49:08 #479 INFO] Running EventBasedCalculator with concurrent_tasks = 32
[2023-12-21 10:49:08 #479 INFO] Checksum of the inputs: 328188081 (total size 436.36 KB)
[2023-12-21 10:49:08 #479 INFO] Extracting the hazard sites from the site model
[2023-12-21 10:49:08 #479 INFO] Read N=8280 hazard sites and L=1 hazard levels
[2023-12-21 10:49:08 #479 INFO] Reading station data from /Users/nicolepaul/Code/earthquake-scenarios/Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/stationlist_seismic.csv
[2023-12-21 10:49:08 #479 INFO] min_iml=[1.e-10]
[2023-12-21 10:49:08 #479 INFO] Reordering the ruptures and storing the events
[2023-12-21 10:49:08

  id | name
1122 | Events
1123 | Full Report
1124 | Ground Motion Fields
1125 | Realizations
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_474.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_475.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_476.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_477.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_478.csv
File saved to ../Algeria/20030521_M6.8_Boumerdes/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_479.csv


### Stations: All (i.e., seismic and macroseismic stations)

In [10]:
# # Job.ini to be use in the sensitivity analysis
# job_ini = os.path.join(oq_folder, 'job_stations_all.ini')

# # Run OpenQuake calculations
# jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# # For each calculation, save gmfs in csv (only for PGA)
# for job in jobs:
#     calc_id = job.calc_id
#     df, txt = oq_get_gmfs(calc_id)
#     out_df = write_gmf_csv(df, 'PGA', comment=txt, 
#                            out_path=oq_sens, suffix=calc_id)